In [1]:
library(GUniFrac)

### Simulation

In [2]:
#ecam dataframe
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/ecam_tutorial/ecam_otu.tsv")
#pd dataframe
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/PD/pd_otu.tsv")
#american_gut dataframe
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/american_gut/american_gut_otu.tsv")
#IDB dataframe
file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/IBD/ibd_otu.tsv")
#Study_12021
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/SIFM/sifm_otu.tsv")


#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/ecam_10245/ecam_otu_table.tsv",
#                "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/pd_10483_otu_table.tsv",
#               "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/american_gut_10317/american_gut_otu_table.tsv",
 #              "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/IBD_1629/1629_otu_table.tsv",
 #               "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/study_12021/12021_otu_table.tsv")
list_of_dfs <- lapply(file_paths, function(file) read.table(file, sep = '\t', header=TRUE, row.names=1))

In [3]:
list_of_dfs <- lapply(file_paths, function(file) read.table(file, sep = '\t', header=TRUE, row.names=1))


In [1]:
#list_of_dfs

In [11]:
names(list_of_dfs) <- c("ecam", "pd","american_gut","IDB","study_12021")


In [5]:
#sample_size=list(length(rownames(list_of_dfs$ecam)),length(rownames(list_of_dfs$pd)))
#sample_size=list(1000,2000,3000,4050,5100,6150,7200,8000,9000,10000) # for pd dataframe
#sample_size=list(1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000) # for ecam dataframe
sample_size=list(4000,5000,6000,6500,7000,7500,8000,8500,9000) # for IDB dataframe
#sample_size=list(1000,1100,1200,1400,1600,1700,1800,1900,2500) # for study_12021
#sample_size=list(1200,2500,3000,4500,5500,6500,7500,8400,9500,11000,12000,13000,14000,15000,16000,17000) # for for american_gut dataframe

In [5]:
sample_size[[2]]

[1] 2000

In [79]:
nsample=list(length(list_of_dfs$ecam),length(list_of_dfs$pd))

In [7]:
nsample[[2]]

[1] 410

In [ ]:
num_dfs <- length(list_of_dfs)
num_sizes <- length(sample_size)
simulation <- vector("list", num_dfs)

for(i in 1:length(list_of_dfs)) {
    simulation[[i]] <- vector("list", num_sizes)
    for(j in 1:length(sample_size)){
    simulation[[i]][[j]] <- SimulateMSeq(
ref.otu.tab = list_of_dfs[[i]], nSam = 600, nOTU = sample_size[[j]],
# True signal setting
diff.otu.pct = 0.1, diff.otu.direct = c("unbalanced"), 
diff.otu.mode = c("abundant"),
covariate.type = c("binary"), grp.ratio = 1,
covariate.eff.mean = 1.0, covariate.eff.sd = 0,
# Confounder signal setting
confounder.type = c("both"), conf.cov.cor = 0.6,
conf.diff.otu.pct = 0.1, conf.nondiff.otu.pct = 0.1,
confounder.eff.mean = 1.0, confounder.eff.sd = 0,
# Depth setting
depth.mu = 10000, depth.theta = 5, depth.conf.factor = 0
)
 }   
}


Iteration 1: Log-likelihood value: -440439138.484338

Iteration 2: Log-likelihood value: -440378659.346065

Iteration 3: Log-likelihood value: -440322458.056214

Iteration 4: Log-likelihood value: -440273955.055712

Iteration 5: Log-likelihood value: -440233651.996686

Iteration 6: Log-likelihood value: -440201316.098619

Iteration 7: Log-likelihood value: -440176167.956054

Iteration 8: Log-likelihood value: -440157215.674671

Iteration 9: Log-likelihood value: -440143511.129051

Iteration 10: Log-likelihood value: -440134110.503762

Iteration 11: Log-likelihood value: -440128120.682499

Iteration 12: Log-likelihood value: -440124825.641028

Iteration 13: Log-likelihood value: -440123460.192069

Iteration 14: Log-likelihood value: -440123111.543757

Iteration 15: Log-likelihood value: -440123071.500689

Iteration 16: Log-likelihood value: -440123070.070577

Iteration 17: Log-likelihood value: -440123070.0398

Iteration 18: Log-likelihood value: -440123070.039181

Iteration 19: Log-lik

In [54]:
sim_american_gut=simulation


In [2]:
#simulation

In [ ]:

saveRDS(simulation, file = "~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/simulated_data/ibd_otu.rds")


In [2]:
loaded_list <- readRDS("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/simulated_data/american_gut_otu.rds")


In [ ]:
loaded_list

### Method run

In [8]:
library(ANCOMBC)

In [9]:
run_method<- function(raw_data, DA_method){

  if(DA_method=="ANCOM-BC"){
    out<- run_ancombc(raw_data = raw_data)
  } else if(DA_method=="FlashWeave"){
    out<- run_flashweave(raw_data = raw_data)
  
  } else if(DA_method=="ANCOM-BC2"){
     out<- run_ancombc2(raw_data = raw_data) 
  }

  return(out)
}


run_flashweave<- function(raw_data){

    setwd("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark")
    # data file paths
    original_data<-"~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/otu_pd_qiime.tsv"
    simulated_data<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.txt"
    biom_file <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.biom'
    output_qza <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.qza'
    metadata<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/meta_table1.tsv"
    output_flash <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/flashnet1.qza'
    neighbour='/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/neighbours1.qza'
    exported <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/exported_neighbours'
    ppd_otu_filtered_qza="~/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_pd_qiime_filtered.qza"
    
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
     meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    #otu data processing for qiime2 input
    otu <- cbind(rownames(otu.tab.sim), otu.tab.sim[,2:length(colnames(otu.tab.sim))])
    colnames(otu)[1] <- "featureid"
    rownames(otu) <- NULL

    write.table(otu, simulated_data, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)
    #metadata processing for qiime2 input
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    metad  <- cbind(rownames(meta.dat),meta.dat)
    colnames(metad)[1] <- "sampleid"
    rownames(metad) <- NULL
    write.table(metad, metadata, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

   # convert data into biom format
    system(paste("biom convert",
                 "-i",simulated_data ,
                 "-o", biom_file ,
                 "--table-type=\"OTU table\"",
                 "--to-hdf5"), intern=TRUE)

    #import data into qiime 
    system(paste("qiime tools import",
             "--input-path", biom_file,
             "--type 'FeatureTable[Frequency]'",
             "--input-format BIOMV210Format",
             "--output-path", output_qza), intern=TRUE)
    
    # filtering data
    system(paste("qiime feature-table filter-features",
             "--i-table", output_qza,
             "--p-min-frequency",50,
             "--p-min-samples",4,
             "--o-filtered-table", ppd_otu_filtered_qza), intern=TRUE)
  
    # Run FlashWeave with sampled parameters
    system(paste("qiime makarsa flashweave",
             "--i-table", ppd_otu_filtered_qza,
             "--m-metadata-file",metadata,
             "--p-heterogeneous", TRUE,         #need to replace parameters with their best 
             "--p-sensitive", TRUE,             #value found in hyperparameter tuning.
             "--p-max-k", 0,
             "--p-alpha", 0.04 ,
             "--o-network", output_flash), intern=TRUE)                              
  
 
 
    system(paste("qiime makarsa list-neighbours",
                "--i-network",output_flash,
                "--p-feature-id", "covariate",
                 "--p-radius",2,
                "--o-neighbours",neighbour), intern=TRUE)                 

   # Export the results from QIIME2
   system(paste("qiime tools export", 
              "--input-path",neighbour,
             "--output-path",exported), intern=TRUE)                
  
  
  
  # Construct file path for metadata.tsv
   metadata_tsv <- paste0(exported, "/metadata.tsv")
  
  # Check if the file exists before reading
  if (!file.exists(metadata_tsv)) {
    cat("Expected metadata.tsv does not exist at path:", metadata_tsv, "\n")
    next
  }
  
  # Read the results
  neighbors <-read.csv(file = metadata_tsv, sep = '\t', row.names = 1)
  
  # take only necessary data
    neighbors=neighbors[-1,]
  row_names_df_to_remove=c("cofounder1","cofounder2")
  neighbors=neighbors[!(row.names(neighbors) %in% row_names_df_to_remove),]
  e=(rownames(otu.tab.sim)%in%rownames(neighbors))   
  pred=data.frame("pred" =ifelse(e, "DA", "NOT_DA") , row.names =rownames(otu.tab.sim) , stringsAsFactors = F)
  df=cbind(pred,tr) 
  bet1=neighbors["Betweenness.Centrality"]
  bet=data.frame("Betweenness.Centrality" =rep(0,length(rownames(otu.tab.sim)[!e])),  row.names =rownames(otu.tab.sim)[!e] , stringsAsFactors = F)
  bet2=rbind(bet1,bet)
  out_df=cbind(df,bet2)
    
    #neighbours=  neighbours[-1,] 
    #e=rownames(otu.tab.sim)%in%rownames(neighbours)   
    #pred=data.frame("pred" =ifelse(e, "DA", "NOT_DA") , row.names =rownames(otu.tab.sim) , stringsAsFactors = F)
    #out_df=cbind(pred,tr) 
    
  # Return a list with the output of the method and the running time:
  return("method_out"=out_df)
}



run_ancombc<- function(raw_data){

    
    setwd("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark")
    # data file paths
    original_data<-"~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/otu_pd_qiime.tsv"
    simulated_data<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.txt"
    biom_file <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.biom'
    output_qza <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_table1.qza'
    metadata<-"/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/meta_table1.tsv"
    ancomoutput <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/ancombc.qza'
    exported <- '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/exported_ancompd'
    ppd_otu_filtered_qza="~/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark/otu_pd_qiime_filtered.qza"
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
    meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    #otu data processing for qiime2 input
    otu <- cbind(rownames(otu.tab.sim), otu.tab.sim[,2:length(colnames(otu.tab.sim))])
    colnames(otu)[1] <- "featureid"
    rownames(otu) <- NULL

    write.table(otu, simulated_data, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)
    #metadata processing for qiime2 input
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    metad  <- cbind(rownames(meta.dat),meta.dat)
    colnames(metad)[1] <- "sampleid"
    rownames(metad) <- NULL
    write.table(metad, metadata, sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

   # convert data into biom format
    system(paste("biom convert",
                 "-i",simulated_data ,
                 "-o", biom_file ,
                 "--table-type=\"OTU table\"",
                 "--to-hdf5"), intern=TRUE)
    #import data into qiime2                
    system(paste("qiime tools import",
             "--input-path", biom_file,
             "--type 'FeatureTable[Frequency]'",
             "--input-format BIOMV210Format",
             "--output-path", output_qza), intern=TRUE)
    
    #ppd_otu_filtered_qza="~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/PD/otu_pd_qiime_filtered.qza"
    system(paste("qiime feature-table filter-features",
             "--i-table", output_qza,
             "--p-min-frequency",50,
             "--p-min-samples",4,
             "--o-filtered-table", ppd_otu_filtered_qza), intern=TRUE)
                   
   # Run ANCOM-BC with sampled parameters
    system(paste("qiime composition ancombc",
             "--i-table",ppd_otu_filtered_qza,
             "--m-metadata-file", metadata,
             "--p-formula", "covariate",
             "--p-p-adj-method", "BH",
            # "--p-zero-cut", 0.7,
            # "--p-lib-cut", 0,
            # "--p-struct-zero", FALSE,
             "--p-neg-lb", FALSE,
             "--p-tol", 0.01,
             "--p-max-iter", 75,
             "--p-conserve", FALSE,
             #"--p-alpha", 0.01,
            # "--p-global", TRUE,
             "--o-differentials", ancomoutput))                
  
   # Export the results from QIIME2
   system(paste("qiime tools export", 
              "--input-path",ancomoutput,
             "--output-path",exported))                
  
  # Construct file path for q_val_slice.csv
   q_val_csv <- paste0(exported, "/q_val_slice.csv")
  
  # Check if the file exists before reading
  if (!file.exists(q_val_csv)) {
    cat("Expected q_val_slice.csv does not exist at path:", q_val_csv, "\n")
    next
  }
  
    # Read the q-value results
    ancom_q_val <- read.csv(file = q_val_csv, sep = ',', row.names = 1)

    # take only necessary data
    ancom_q_val=data.frame("covariateunhealthy" = ancom_q_val$covariateunhealthy, row.names = rownames(ancom_q_val), stringsAsFactors = F)
    #construct output dataframe
    e=!(rownames(otu.tab.sim)%in%rownames(ancom_q_val))
    nq=rownames(otu.tab.sim)[e]
    nqv=data.frame("covariateunhealthy" = rep(NA, length(nq)), row.names = nq, stringsAsFactors = F)
    q_val_all=rbind(ancom_q_val,nqv)
    names(q_val_all)[names(q_val_all) == "covariateunhealthy"] <- "q_value"
    pred= ifelse(ancom_q_val < 0.05, "DA", "NOT_DA")
    pred=as.data.frame(pred)
    d=!(rownames(otu.tab.sim)%in%rownames(pred))
    npred=rownames(otu.tab.sim)[d]
    ND=data.frame("covariateunhealthy" = rep("NOT_DA", length(npred)), row.names = npred, stringsAsFactors = F)
    pr=rbind(pred,ND)
    out_df=cbind(pr,q_val_all,tr)
    names(out_df)[names(out_df) == "covariateunhealthy"] <- "pred"

  return("method_out"=out_df)
}


run_ancombc2<- function(raw_data){

    library(phyloseq)
    meta.dat <- data.frame(X = raw_data$covariate, Z1 = raw_data$confounder[, 1],
                       Z2 = raw_data$confounder[, 2])
    otu.tab.sim <- raw_data$otu.tab.sim
    actual=ifelse(raw_data$diff.otu.ind, "DA", "NOT_DA")
    tr=data.frame("truth" = actual, row.names = rownames(otu.tab.sim), stringsAsFactors = F)
    meta.dat$X=ifelse(meta.dat$X, "healthy","unhealthy")
    colnames(meta.dat)=c("covariate", "cofounder1","cofounder2")

    otu<-otu_table(otu.tab.sim,taxa_are_rows = TRUE)
    sam<-sample_data(meta.dat)
    physeq = phyloseq(otu, sam)
                   
   # Run ANCOM-BC2 with sampled parameters
    set.seed(123)
    output = ancombc2(data = physeq, assay_name = "counts", tax_level = "Species",
                  fix_formula = "covariate", rand_formula = NULL,
                  p_adj_method = "holm", pseudo_sens = TRUE,
                  prv_cut = 0.10, lib_cut = 1000, s0_perc = 0.05,
                  group = "covariate", struc_zero = FALSE, neg_lb = FALSE,
                  alpha = 0.05, n_cl = 2, verbose = TRUE,
                  global = FALSE, pairwise = TRUE, 
                  dunnet = FALSE, trend = FALSE,
                  iter_control = list(tol = 1e-5, max_iter = 20, 
                                      verbose = FALSE),
                  em_control = list(tol = 1e-5, max_iter = 100),
                  lme_control = NULL, 
                  mdfdr_control = list(fwer_ctrl_method = "holm", B = 100), 
                  trend_control = NULL)

     res_prim = output$res
     #res_pair = output$res_pair
     rownames(res_prim)=res_prim$taxon
     res_prim$taxon=NULL

    # take only necessary data
    ancom_q_val=data.frame("covariateunhealthy" = res_prim$q_covariateunhealthy, row.names = rownames(res_prim), stringsAsFactors = F)
    #construct output dataframe
    e=!(rownames(otu.tab.sim)%in%rownames(ancom_q_val))
    nq=rownames(otu.tab.sim)[e]
    nqv=data.frame("covariateunhealthy" = rep(NA, length(nq)), row.names = nq, stringsAsFactors = F)
    q_val_all=rbind(ancom_q_val,nqv)
    names(q_val_all)[names(q_val_all) == "covariateunhealthy"] <- "q_value"
    pred= ifelse(ancom_q_val < 0.05, "DA", "NOT_DA")
    pred=as.data.frame(pred)
    d=!(rownames(otu.tab.sim)%in%rownames(pred))
    npred=rownames(otu.tab.sim)[d]
    ND=data.frame("covariateunhealthy" = rep("NOT_DA", length(npred)), row.names = npred, stringsAsFactors = F)
    pr=rbind(pred,ND)
    out_df=cbind(pr,q_val_all,tr)
    names(out_df)[names(out_df) == "covariateunhealthy"] <- "pred"
    
  return("method_out"=out_df)
}



### Running methods on simulated data

In [10]:
#simulation <- readRDS("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/simulated_data/sifm_otu.rds")
simulation <- readRDS("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/simulated_data/pd_otu.rds")
#simulation <- readRDS("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/simulated_data/ibd_otu.rds")
#simulation <- readRDS("/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/simulated_data/ecam_otu.rds")
#simulation <- readRDS("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/simulated_data/american_gut_otu.rds")

In [3]:
#simulation[[1]][[1]]

In [ ]:
#sample number
#american_gut=500
#pd=350
#ecam=400
#sifm=250
#ibd=600

In [11]:
#sample_size=list(length(rownames(list_of_dfs$ecam)),length(rownames(list_of_dfs$pd)))
sample_size=list(1000,2000,3000,4050,5100,6150,7200,8000,9000,10000) # for pd dataframe
#sample_size=list(1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000) # for ecam dataframe
#sample_size=list(4000,5000,6000,6500,7000,7500,8000,8500,9000) # for IDB dataframe
#sample_size=list(1000,1100,1200,1400,1600,1700,1800,1900,2500) # for study_12021
#sample_size=list(1200,2500,3000,4500,5500,6500,7500,8400,9500,11000,12000,13000,14000,15000,16000,17000) # for for american_gut dataframe

In [12]:
#ecam dataframe
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/ecam_tutorial/ecam_otu.tsv")
#pd dataframe
file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/PD/pd_otu.tsv")
#american_gut dataframe
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/american_gut/american_gut_otu.tsv")
#IDB dataframe
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/IBD/ibd_otu.tsv")
#Study_12021
#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/data/SIFM/sifm_otu.tsv")


#file_paths <- c("~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/ecam_10245/ecam_otu_table.tsv",
#                "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/PD_10483/pd_10483_otu_table.tsv",
#               "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/american_gut_10317/american_gut_otu_table.tsv",
 #              "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/IBD_1629/1629_otu_table.tsv",
 #               "~/from_pendrive/PhD/DAA_benchmark_study/qiime_ancombc/data/study_12021/12021_otu_table.tsv")
list_of_dfs <- lapply(file_paths, function(file) read.table(file, sep = '\t', header=TRUE, row.names=1))

In [ ]:
# List of methods you want to run
methods_to_run <- c("FlashWeave")  # Add more if needed , "FlashWeave"  "ANCOM-BC"

# Initialize results_list
results_list <- list()
for(method in methods_to_run) {
  results_list[[method]] <- vector("list", length(list_of_dfs))
  for(i in 1:length(list_of_dfs)) {
    results_list[[method]][[i]] <- vector("list", length(sample_size))
  }
}

# Loop over datasets and methods
for(i in 1:length(list_of_dfs)) {
  for(j in 1:length(sample_size)) {
    data <- simulation[[i]][[j]]
    
    for(method in methods_to_run) {
      # Run the method
      result <- run_method(data, method)
      
      # Store the result
      results_list[[method]][[i]][[j]] <- result
    }
  }
}


In [287]:
library(ANCOMBC)
library(phyloseq)

In [226]:
#results_list

### RUNNING OUTPUT TO GENERATE PR-RECALL DATAFRAME

In [122]:
# Compute Precision and Recall
# param:
#   - truth: a vector that contains the labels that identify the simulated otu as "DA" or "NOT_DA".
#   - pred: a vector that contains the predicted labels.
# return: a named vector with precision and recall

compute_precision_recall<- function(truth, pred){

  if(all(is.na(pred))){
    out<- c(NA,NA)
    names(out)<- c("recall", "precision")
    return(out)
  }

  # Reassign the labels. The positive class is the one with the lowest number, i.e. "DA":
  truth<- ifelse(truth == "DA", 1, 0)
  pred<- ifelse(pred == "DA", 1, 0)

  # Compute the number of False Positive (FP), False Negative (FN), True Positive (TP), True Negative (TN):
  FP<- sum(truth==0 & pred==1)
  FN<- sum(truth==1 & pred==0)
  TP<- sum(truth==1 & pred==1)
  TN<- sum(truth==0 & pred==0)

  # Calculate Precision and Recall:
  if(TP==0 & FP==0){
    precision<- NA
  }else{
    precision<- TP/(TP+FP)
  }

  if(TP==0 & FN==0){
    recall<- NA
  }else{
    recall<- TP/(TP+FN)
  }

  if(FP==0 & TN==0){
    fpr<- NA
  }else{
    fpr<- FP/(FP+TN)
  }
    
  if(TP==0 & FP==0){
    fdr<- NA
  }else{
    fdr<- FP/(TP+FP)
  }  
     
    
  f1_score<- 2*(recall*precision)/(precision+recall)

  # Return precision and recall:
  out<- c(recall,precision,fpr,fdr, f1_score)
  names(out)<- c("recall", "precision","fpr","fdr", "f1_score")
  return(out)
}


In [140]:
# Initialize a dataframe to store precision-recall metrics
precision_recall_df <- data.frame()

for(method in methods_to_run) {
    recall_vector <- c()
    precision_vector <- c()
    fpr_vector <- c()
    fdr_vector <- c()
    f1_vector <- c()
    
    for(i in 1:length(list_of_dfs)) {
        for(j in 1:length(sample_size)) {
            method_results <- results_list[[method]][[i]][[j]]
            
            prec_rec_out <- compute_precision_recall(truth = method_results$truth, pred = method_results$pred)
            
            recall_vector <- c(recall_vector, prec_rec_out["recall"])
            precision_vector <- c(precision_vector, prec_rec_out["precision"])
            fpr_vector <- c(fpr_vector, prec_rec_out["fpr"])
            fdr_vector <- c(fdr_vector, prec_rec_out["fdr"])
            f1_vector <- c(f1_vector, prec_rec_out["f1_score"])
        }
    }
    
    tmp_df <- data.frame("method" = rep(method, length(recall_vector)),
                         "recall" = recall_vector,
                         "precision" = precision_vector,
                         "fpr" = fpr_vector,
                         "fdr" = fdr_vector,
                         "F1" = f1_vector,
                         "na_precision" = rep(sum(is.na(precision_vector)), length(recall_vector)),
                         stringsAsFactors = F)
    
    precision_recall_df <- rbind(precision_recall_df, tmp_df)
}


In [4]:
#precision_recall_df

In [142]:
write.table(precision_recall_df, '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave2_pdf.tsv', sep = "\t", col.names=TRUE, row.names=FALSE, quote = FALSE)

In [1]:
ancom_american_gut<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancom_american_gut.tsv', sep = '\t')
ancombc2_american_gut<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancombc2_american_gut.tsv', sep = '\t')
flashweave_radius4_american_gut<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius4_american_gut.tsv', sep = '\t')
flashweave_radius2_american_gut<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius2_american_gut.tsv', sep = '\t')
flashweave_radius3_american_gut<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius3_american_gut.tsv', sep = '\t')


In [5]:
#ancombc2_american_gut

In [2]:
flashweave_radius4_american_gut$"na_precision"=NULL
flashweave_radius2_american_gut$"na_precision"=NULL
flashweave_radius3_american_gut$"na_precision"=NULL
ancom_american_gut$"na_precision"=NULL
ancombc2_american_gut$"na_precision"=NULL

In [3]:
names(flashweave_radius3_american_gut)=c('method3','recall3','precision3','fpr3','fdr3','F13')
names(flashweave_radius4_american_gut)=c('method4','recall4','precision4','fpr4','fdr4','F14')


In [4]:
combine_flashweave_am=cbind(flashweave_radius2_american_gut,flashweave_radius3_american_gut,flashweave_radius4_american_gut)

In [5]:
combine_flashweave_am$method3=NULL
combine_flashweave_am$method4=NULL

In [6]:
#combine_flashweave_am

In [7]:
tmp_df_am <- data.frame( "recall3" = rep(0, length(ancom_american_gut$recall)),
                      "precision3" = rep(0, length(ancom_american_gut$recall)),
                      "fpr3" = rep(0, length(ancom_american_gut$recall)),
                      "fdr3" = rep(0, length(ancom_american_gut$recall)),
                      "F13" = rep(0, length(ancom_american_gut$recall)),
                     "recall4" = rep(0, length(ancom_american_gut$recall)),
                      "precision4" = rep(0, length(ancom_american_gut$recall)),
                      "fpr4" = rep(0, length(ancom_american_gut$recall)),
                      "fdr4" = rep(0, length(ancom_american_gut$recall)),
                      "F14" = rep(0, length(ancom_american_gut$recall)),
                         stringsAsFactors = F)

In [8]:
tdf_am=cbind(ancom_american_gut,tmp_df_am)

In [9]:
tdf_ambc2=cbind(ancombc2_american_gut,tmp_df_am)

In [10]:
combine_american_gut=rbind(combine_flashweave_am,tdf_am,tdf_ambc2)

In [11]:
combine_american_gut <- combine_american_gut[, c(1, 2,7,12, 3,8,13, 4,9,14, 5,10,15,6,11,16)]

In [12]:
#combine_american_gut

In [13]:
ancom_IDB<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancom_ibd.tsv', sep = '\t')
ancombc2_IDB<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancombc2_ibd.tsv', sep = '\t')
flashweave_radius4_IDB<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius4_ibd.tsv', sep = '\t')
flashweave_radius2_IDB<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius2_ibd.tsv', sep = '\t')
flashweave_radius3_IDB<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius3_ibd.tsv', sep = '\t')

In [14]:
flashweave_radius4_IDB$"na_precision"=NULL
flashweave_radius2_IDB$"na_precision"=NULL
flashweave_radius3_IDB$"na_precision"=NULL
ancom_IDB$"na_precision"=NULL
ancombc2_IDB$"na_precision"=NULL

In [15]:
names(flashweave_radius3_IDB)=c('method3','recall3','precision3','fpr3','fdr3','F13')
names(flashweave_radius4_IDB)=c('method4','recall4','precision4','fpr4','fdr4','F14')

In [16]:
combine_flashweave_ibd=cbind(flashweave_radius2_IDB,flashweave_radius3_IDB,flashweave_radius4_IDB)

In [17]:
combine_flashweave_ibd$method3=NULL
combine_flashweave_ibd$method4=NULL

In [18]:
tmp_df_ibd <- data.frame( "recall3" = rep(0, length(ancom_IDB$recall)),
                      "precision3" = rep(0, length(ancom_IDB$recall)),
                      "fpr3" = rep(0, length(ancom_IDB$recall)),
                      "fdr3" = rep(0, length(ancom_IDB$recall)),
                      "F13" = rep(0, length(ancom_IDB$recall)),
                     "recall4" = rep(0, length(ancom_IDB$recall)),
                      "precision4" = rep(0, length(ancom_IDB$recall)),
                      "fpr4" = rep(0, length(ancom_IDB$recall)),
                      "fdr4" = rep(0, length(ancom_IDB$recall)),
                      "F14" = rep(0, length(ancom_IDB$recall)),
                         stringsAsFactors = F)

In [19]:
tdf_ibd=cbind(ancom_IDB,tmp_df_ibd)

In [20]:
tdfbc2_ibd=cbind(ancombc2_IDB,tmp_df_ibd)

In [21]:
combine_IDB=rbind(combine_flashweave_ibd,tdf_ibd,tdfbc2_ibd)

In [22]:
combine_IDB <- combine_IDB[, c(1, 2,7,12, 3,8,13, 4,9,14, 5,10,15,6,11,16)]

In [6]:
#combine_IDB

In [24]:
ancom_ecam<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancom_ecamf.tsv', sep = '\t')
ancombc2_ecam<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancombc2_ecamf.tsv', sep = '\t')
flashweave_radius4_ecam<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius4_ecamf.tsv', sep = '\t')
flashweave_radius2_ecam<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius2_ecamf.tsv', sep = '\t')
flashweave_radius3_ecam<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius3_ecamf.tsv', sep = '\t')

In [25]:
flashweave_radius4_ecam$"na_precision"=NULL
flashweave_radius2_ecam$"na_precision"=NULL
flashweave_radius3_ecam$"na_precision"=NULL
ancom_ecam$"na_precision"=NULL
ancombc2_ecam$"na_precision"=NULL

In [26]:
names(flashweave_radius3_ecam)=c('method3','recall3','precision3','fpr3','fdr3','F13')
names(flashweave_radius4_ecam)=c('method4','recall4','precision4','fpr4','fdr4','F14')

In [27]:
combine_flashweave_ecam=cbind(flashweave_radius2_ecam,flashweave_radius3_ecam,flashweave_radius4_ecam)

In [28]:
combine_flashweave_ecam$method3=NULL
combine_flashweave_ecam$method4=NULL

In [29]:
tmp_df_ecam <- data.frame( "recall3" = rep(0, length(ancom_ecam$recall)),
                      "precision3" = rep(0, length(ancom_ecam$recall)),
                      "fpr3" = rep(0, length(ancom_ecam$recall)),
                      "fdr3" = rep(0, length(ancom_ecam$recall)),
                      "F13" = rep(0, length(ancom_ecam$recall)),
                     "recall4" = rep(0, length(ancom_ecam$recall)),
                      "precision4" = rep(0, length(ancom_ecam$recall)),
                      "fpr4" = rep(0, length(ancom_ecam$recall)),
                      "fdr4" = rep(0, length(ancom_ecam$recall)),
                      "F14" = rep(0, length(ancom_ecam$recall)),
                         stringsAsFactors = F)

In [30]:
tdf_ecam=cbind(ancom_ecam,tmp_df_ecam)

In [31]:
tdfbc2_ecam=cbind(ancombc2_ecam,tmp_df_ecam)

In [32]:
combine_ecam=rbind(combine_flashweave_ecam,tdf_ecam,tdfbc2_ecam)

In [33]:
combine_ecam <- combine_ecam[, c(1, 2,7,12, 3,8,13, 4,9,14, 5,10,15,6,11,16)]

In [34]:
#combine_ecam 

In [145]:
ancom_pd<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancombc_pdf.tsv', sep = '\t')
ancombc2_pd<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancombc2_pd.tsv', sep = '\t')
flashweave_radius2_pd<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius2_pd.tsv', sep = '\t')
flashweave_radius3_pd<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_redius3_pd.tsv', sep = '\t')
flashweave_radius4_pd<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius4_pd.tsv', sep = '\t')

In [146]:

flashweave_radius4_pd$"na_precision"=NULL
flashweave_radius2_pd$"na_precision"=NULL
flashweave_radius3_pd$"na_precision"=NULL
ancom_pd$"na_precision"=NULL
ancombc2_pd$"na_precision"=NULL

In [147]:
names(flashweave_radius3_pd)=c('method3','recall3','precision3','fpr3','fdr3','F13')
names(flashweave_radius4_pd)=c('method4','recall4','precision4','fpr4','fdr4','F14')

In [148]:
combine_flashweave_pd=cbind(flashweave_radius2_pd,flashweave_radius3_pd,flashweave_radius4_pd)

In [149]:
combine_flashweave_pd$method3=NULL
combine_flashweave_pd$method4=NULL

In [150]:
tmp_df_pd <- data.frame( "recall3" = rep(0, length(ancom_pd$recall)),
                      "precision3" = rep(0, length(ancom_pd$recall)),
                      "fpr3" = rep(0, length(ancom_pd$recall)),
                      "fdr3" = rep(0, length(ancom_pd$recall)),
                      "F13" = rep(0, length(ancom_pd$recall)),
                     "recall4" = rep(0, length(ancom_pd$recall)),
                      "precision4" = rep(0, length(ancom_pd$recall)),
                      "fpr4" = rep(0, length(ancom_pd$recall)),
                      "fdr4" = rep(0, length(ancom_pd$recall)),
                      "F14" = rep(0, length(ancom_pd$recall)),
                         stringsAsFactors = F)

In [151]:
tdf_pd=cbind(ancom_pd,tmp_df_pd)

In [42]:
tdfbc2_pd=cbind(ancombc2_pd,tmp_df_pd)

In [153]:
combine_pd=rbind(combine_flashweave_pd,tdf_pd,tdfbc2_pd)

In [154]:
combine_pd <- combine_pd[, c(1, 2,7,12, 3,8,13, 4,9,14, 5,10,15,6,11,16)]

In [5]:
#combine_pd

In [46]:
ancom_study_12021=read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancom_sifm.tsv', sep = '\t')
ancombc2_study_12021=read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_ancombc2_sifm.tsv', sep = '\t')
flashweave_radius4_study_12021<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius4_sifm.tsv', sep = '\t')
flashweave_radius2_study_12021<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius2_sifm.tsv', sep = '\t')
flashweave_radius3_study_12021<- read.csv(file = '/home/zakir/from_pendrive/PhD/DAA_benchmark_study/Benchmark_study_makarsa/result/new_result/output_dataframe_flashweave_radius3_sifm.tsv', sep = '\t')

In [47]:
flashweave_radius4_study_12021$"na_precision"=NULL
flashweave_radius2_study_12021$"na_precision"=NULL
flashweave_radius3_study_12021$"na_precision"=NULL
ancom_study_12021$"na_precision"=NULL
ancombc2_study_12021$"na_precision"=NULL

In [48]:
names(flashweave_radius3_study_12021)=c('method3','recall3','precision3','fpr3','fdr3','F13')
names(flashweave_radius4_study_12021)=c('method4','recall4','precision4','fpr4','fdr4','F14')

In [49]:
combine_flashweave_sifm=cbind(flashweave_radius2_study_12021,flashweave_radius3_study_12021,flashweave_radius4_study_12021)

In [50]:
combine_flashweave_sifm$method3=NULL
combine_flashweave_sifm$method4=NULL

In [51]:
tmp_df_sifm <- data.frame( "recall3" = rep(0, length(ancom_study_12021$recall)),
                      "precision3" = rep(0, length(ancom_study_12021$recall)),
                      "fpr3" = rep(0, length(ancom_study_12021$recall)),
                      "fdr3" = rep(0, length(ancom_study_12021$recall)),
                      "F13" = rep(0, length(ancom_study_12021$recall)),
                     "recall4" = rep(0, length(ancom_study_12021$recall)),
                      "precision4" = rep(0, length(ancom_study_12021$recall)),
                      "fpr4" = rep(0, length(ancom_study_12021$recall)),
                      "fdr4" = rep(0, length(ancom_study_12021$recall)),
                      "F14" = rep(0, length(ancom_study_12021$recall)),
                         stringsAsFactors = F)

In [52]:
tdf_sifm=cbind(ancom_study_12021,tmp_df_sifm)

In [53]:
tdfbc2_sifm=cbind(ancombc2_study_12021,tmp_df_sifm)

In [54]:
combine_study_12021=rbind(combine_flashweave_sifm,tdf_sifm,tdfbc2_sifm)

In [55]:
combine_study_12021 <- combine_study_12021[, c(1, 2,7,12, 3,8,13, 4,9,14, 5,10,15,6,11,16)]

In [56]:
#combine_study_12021

### Final Plot

In [94]:
precision_recall_df=combine_study_12021
#precision_recall_df=combine_american_gut
#precision_recall_df=combine_IDB
#precision_recall_df=combine_ecam
#precision_recall_df=combine_pd

In [95]:
png("Box_plot_sifmf.png")
library(ggplot2)
library(dplyr)
library(forcats) # For releveling factors
library(viridis) # For color scales

df <- precision_recall_df

# Convert the data to a long format for easier plotting with ggplot2
df_long <- pivot_longer(df, cols = -c(method), names_to = "metric", values_to = "value")

# Filter the dataframe if necessary, using your custom conditions
ibd_long <- df_long %>%
  filter(!(method == "ANCOM-BC2" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4")))) %>%
  filter(!(method == "ANCOM-BC" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4"))))

# Adding a 'radius' column based on the conditions provided
ibd_long <- ibd_long %>%
  mutate(radius = ifelse(method == "FlashWeave",
                         ifelse(grepl("3$", metric), " radius 3", 
                         ifelse(grepl("4$", metric), " radius 4", " radius 2")),
                         ""))

# Removing trailing 3s or 4s from the metric column
ibd_long$metric <- gsub("3$", "", gsub("4$", "", ibd_long$metric))

# Reordering the levels of metric to ensure correct order in the plot
ibd_long$metric <- fct_relevel(ibd_long$metric, "F1", "precision", "recall", "fdr", "fpr")

# Now plot the data using ggplot2 with viridis color scale
ggplot(ibd_long, aes(x = metric, y = value, fill = interaction(method, radius, sep = ""))) +
  geom_boxplot() +
  scale_fill_viridis_d() +
  theme(
    axis.title.x = element_text(face = "bold", size = 12), 
    axis.title.y = element_text(face = "bold", size = 12), 
    axis.text.x = element_text(face = "bold", size = 12 ,angle = 45, hjust = 1),  
    axis.text.y = element_text(face = "bold", size = 12),  
    plot.title = element_text(face = "bold", size = 14)    
  )+
 # theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(x = "Metric", y = "Value", fill = "Method & Radius", title = "         Performance evaluation on SIFM data.",)
dev.off()

png 
  2

In [96]:
#precision_recall_df=combine_study_12021
precision_recall_df=combine_american_gut
#precision_recall_df=combine_IDB
#precision_recall_df=combine_ecam
#precision_recall_df=combine_pd

In [97]:
png("Box_plot_american_gutf.png")
library(ggplot2)
library(dplyr)
library(forcats) # For releveling factors
library(viridis) # For color scales

df <- precision_recall_df

# Convert the data to a long format for easier plotting with ggplot2
df_long <- pivot_longer(df, cols = -c(method), names_to = "metric", values_to = "value")

# Filter the dataframe if necessary, using your custom conditions
ibd_long <- df_long %>%
  filter(!(method == "ANCOM-BC2" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4")))) %>%
  filter(!(method == "ANCOM-BC" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4"))))

# Adding a 'radius' column based on the conditions provided
ibd_long <- ibd_long %>%
  mutate(radius = ifelse(method == "FlashWeave",
                         ifelse(grepl("3$", metric), " radius 3", 
                         ifelse(grepl("4$", metric), " radius 4", " radius 2")),
                         ""))

# Removing trailing 3s or 4s from the metric column
ibd_long$metric <- gsub("3$", "", gsub("4$", "", ibd_long$metric))

# Reordering the levels of metric to ensure correct order in the plot
ibd_long$metric <- fct_relevel(ibd_long$metric, "F1", "precision", "recall", "fdr", "fpr")

# Now plot the data using ggplot2 with viridis color scale
ggplot(ibd_long, aes(x = metric, y = value, fill = interaction(method, radius, sep = ""))) +
  geom_boxplot() +
  scale_fill_viridis_d() +
  theme(
    axis.title.x = element_text(face = "bold", size = 12), 
    axis.title.y = element_text(face = "bold", size = 12), 
    axis.text.x = element_text(face = "bold", size = 12 ,angle = 45, hjust = 1),  
    axis.text.y = element_text(face = "bold", size = 12),  
    plot.title = element_text(face = "bold", size = 14)    
  )+
 # theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(x = "Metric", y = "Value", fill = "Method & Radius", title = "  Performance evaluation on American gut data.",)
dev.off()

png 
  2

In [98]:
#precision_recall_df=combine_study_12021
#precision_recall_df=combine_american_gut
precision_recall_df=combine_IDB
#precision_recall_df=combine_ecam
#precision_recall_df=combine_pd

In [99]:
png("Box_plot_ibdf.png")
library(ggplot2)
library(dplyr)
library(forcats) # For releveling factors
library(viridis) # For color scales

df <- precision_recall_df

# Convert the data to a long format for easier plotting with ggplot2
df_long <- pivot_longer(df, cols = -c(method), names_to = "metric", values_to = "value")

# Filter the dataframe if necessary, using your custom conditions
ibd_long <- df_long %>%
  filter(!(method == "ANCOM-BC2" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4")))) %>%
  filter(!(method == "ANCOM-BC" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4"))))

# Adding a 'radius' column based on the conditions provided
ibd_long <- ibd_long %>%
  mutate(radius = ifelse(method == "FlashWeave",
                         ifelse(grepl("3$", metric), " radius 3", 
                         ifelse(grepl("4$", metric), " radius 4", " radius 2")),
                         ""))

# Removing trailing 3s or 4s from the metric column
ibd_long$metric <- gsub("3$", "", gsub("4$", "", ibd_long$metric))

# Reordering the levels of metric to ensure correct order in the plot
ibd_long$metric <- fct_relevel(ibd_long$metric, "F1", "precision", "recall", "fdr", "fpr")

# Now plot the data using ggplot2 with viridis color scale
ggplot(ibd_long, aes(x = metric, y = value, fill = interaction(method, radius, sep = ""))) +
  geom_boxplot() +
  scale_fill_viridis_d() +
  theme(
    axis.title.x = element_text(face = "bold", size = 12), 
    axis.title.y = element_text(face = "bold", size = 12), 
    axis.text.x = element_text(face = "bold", size = 12 ,angle = 45, hjust = 1),  
    axis.text.y = element_text(face = "bold", size = 12),  
    plot.title = element_text(face = "bold", size = 14)    
  )+
 # theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(x = "Metric", y = "Value", fill = "Method & Radius", title = "        Performance evaluation on IBD data.",)
dev.off()

png 
  2

In [100]:

#precision_recall_df=combine_study_12021
#precision_recall_df=combine_american_gut
#precision_recall_df=combine_IDB
precision_recall_df=combine_ecam
#precision_recall_df=combine_pd


In [101]:
png("Box_plot_ecamf.png")
library(ggplot2)
library(dplyr)
library(forcats) # For releveling factors
library(viridis) # For color scales

df <- precision_recall_df

# Convert the data to a long format for easier plotting with ggplot2
df_long <- pivot_longer(df, cols = -c(method), names_to = "metric", values_to = "value")

# Filter the dataframe if necessary, using your custom conditions
ibd_long <- df_long %>%
  filter(!(method == "ANCOM-BC2" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4")))) %>%
  filter(!(method == "ANCOM-BC" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4"))))

# Adding a 'radius' column based on the conditions provided
ibd_long <- ibd_long %>%
  mutate(radius = ifelse(method == "FlashWeave",
                         ifelse(grepl("3$", metric), " radius 3", 
                         ifelse(grepl("4$", metric), " radius 4", " radius 2")),
                         ""))

# Removing trailing 3s or 4s from the metric column
ibd_long$metric <- gsub("3$", "", gsub("4$", "", ibd_long$metric))

# Reordering the levels of metric to ensure correct order in the plot
ibd_long$metric <- fct_relevel(ibd_long$metric, "F1", "precision", "recall", "fdr", "fpr")

# Now plot the data using ggplot2 with viridis color scale
ggplot(ibd_long, aes(x = metric, y = value, fill = interaction(method, radius, sep = ""))) +
  geom_boxplot() +
  scale_fill_viridis_d() +
  theme(
    axis.title.x = element_text(face = "bold", size = 12), 
    axis.title.y = element_text(face = "bold", size = 12), 
    axis.text.x = element_text(face = "bold", size = 12 ,angle = 45, hjust = 1),  
    axis.text.y = element_text(face = "bold", size = 12),  
    plot.title = element_text(face = "bold", size = 14)    
  )+
 # theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(x = "Metric", y = "Value", fill = "Method & Radius", title = "        Performance evaluation on ECAM data.",)
dev.off()

png 
  2

In [156]:
#precision_recall_df=combine_study_12021
#precision_recall_df=combine_american_gut
#precision_recall_df=combine_IDB
#precision_recall_df=combine_ecam
precision_recall_df=combine_pd



In [158]:
png("Box_plot_pdff.png")
library(ggplot2)
library(dplyr)
library(forcats) # For releveling factors
library(viridis) # For color scales

df <- precision_recall_df

# Convert the data to a long format for easier plotting with ggplot2
df_long <- pivot_longer(df, cols = -c(method), names_to = "metric", values_to = "value")

# Filter the dataframe if necessary, using your custom conditions
ibd_long <- df_long %>%
  filter(!(method == "ANCOM-BC2" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4")))) %>%
  filter(!(method == "ANCOM-BC" & (metric %in% c("F13", "F14", "fdr3", "fdr4", "fpr3", "fpr4", "recall3", "recall4", "precision3", "precision4"))))

# Adding a 'radius' column based on the conditions provided
ibd_long <- ibd_long %>%
  mutate(radius = ifelse(method == "FlashWeave",
                         ifelse(grepl("3$", metric), " radius 3", 
                         ifelse(grepl("4$", metric), " radius 4", " radius 2")),
                         ""))

# Removing trailing 3s or 4s from the metric column
ibd_long$metric <- gsub("3$", "", gsub("4$", "", ibd_long$metric))

# Reordering the levels of metric to ensure correct order in the plot
ibd_long$metric <- fct_relevel(ibd_long$metric, "F1", "precision", "recall", "fdr", "fpr")

# Now plot the data using ggplot2 with viridis color scale
ggplot(ibd_long, aes(x = metric, y = value, fill = interaction(method, radius, sep = ""))) +
  geom_boxplot() +
  scale_fill_viridis_d() +
  theme(
    axis.title.x = element_text(face = "bold", size = 12), 
    axis.title.y = element_text(face = "bold", size = 12), 
    axis.text.x = element_text(face = "bold", size = 12 ,angle = 45, hjust = 1),  
    axis.text.y = element_text(face = "bold", size = 12),  
    plot.title = element_text(face = "bold", size = 14)    
  )+
 # theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(x = "Metric", y = "Value", fill = "Method & Radius", title = "            Performance evaluation on PD data.",)
dev.off()

png 
  2

In [160]:
getwd()

[1] "/home/zakir/from_pendrive/PhD/DAA_benchmark_study/qiime2_benchmark"

### t-test

In [100]:
df=combine_study_12021

In [67]:
#df

In [105]:
precision_recall_df=combine_study_12021
#precision_recall_df=combine_american_gut
#precision_recall_df=combine_IBD
#precision_recall_df=combine_ecam
#precision_recall_df=combine_pd

In [82]:
# List of datasets and their names
datasets <- list(combine_study_12021, combine_american_gut, combine_IDB, combine_ecam, combine_pd)
dataset_names <- c("SIFM", "American_gut", "IBD", "ECAM", "PD")

# List of metrics
metrics <- c("recall", "precision", "fpr", "fdr", "F1")

# Initialize an empty data frame to store the results
results <- data.frame(Dataset = character(),
                      Method1 = character(),
                      Method2 = character(),
                      Metric = character(),
                      T_Value = numeric(),
                      P_Value = numeric(),
                      stringsAsFactors = FALSE)

# Perform t-tests for each dataset and metric
for (d in seq_along(datasets)) {
    dataset <- datasets[[d]]
    dataset_name <- dataset_names[d]
  
  for (metric in metrics) {
  
    # Perform t-test for each pair of methods
    unique_methods <- unique(dataset$method)
    for (i in 1:(length(unique_methods) - 1)) {
      for (j in (i + 1):length(unique_methods)) {
        method1 <- unique_methods[i]
        method2 <- unique_methods[j]
        values1 <- dataset[dataset$method == method1, metric]
        values2 <- dataset[dataset$method == method2, metric]

        # Perform t-test
        t_test <- t.test(values1, values2)

        # Store results in the data frame
        results <- rbind(results, data.frame(Dataset = dataset_name,
                                             Method1 = method1,
                                             Method2 = method2,
                                             Metric = metric,
                                             T_Value = t_test$statistic,
                                             P_Value = t_test$p.value))
      }
    }
  }
}

# Display the results
print(results,row.names = FALSE)



      Dataset    Method1   Method2    Metric    T_Value      P_Value
         SIFM FlashWeave  ANCOM-BC    recall  3.3918487 6.189076e-03
         SIFM FlashWeave ANCOM-BC2    recall  5.5197847 3.072692e-04
         SIFM   ANCOM-BC ANCOM-BC2    recall  4.0024468 1.215161e-03
         SIFM FlashWeave  ANCOM-BC precision -0.2333340 8.188469e-01
         SIFM FlashWeave ANCOM-BC2 precision -0.6542939 5.288652e-01
         SIFM   ANCOM-BC ANCOM-BC2 precision -0.5218486 6.125029e-01
         SIFM FlashWeave  ANCOM-BC       fpr  3.0248496 1.380608e-02
         SIFM FlashWeave ANCOM-BC2       fpr  4.8667543 1.135067e-03
         SIFM   ANCOM-BC ANCOM-BC2       fpr  5.5628575 1.679536e-04
         SIFM FlashWeave  ANCOM-BC       fdr  0.2333340 8.188469e-01
         SIFM FlashWeave ANCOM-BC2       fdr  0.6542939 5.288652e-01
         SIFM   ANCOM-BC ANCOM-BC2       fdr  0.5218486 6.125029e-01
         SIFM FlashWeave  ANCOM-BC        F1  3.6942294 2.456886e-03
         SIFM FlashWeave ANCOM-BC2